In [9]:
import cuml
from cuml.svm import SVC
import numpy as np
from cuml.svm import LinearSVC

In [4]:
from osgeo import gdal
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import geopandas as gpd
import numpy as np
from sklearn import metrics
import os
import matplotlib.pylab as plt
from sklearn.model_selection import (
    TimeSeriesSplit,
    KFold,
    ShuffleSplit,
    StratifiedKFold,
    GroupShuffleSplit,
    GroupKFold,
    StratifiedShuffleSplit,
    StratifiedGroupKFold,
)
from mpl_toolkits import mplot3d
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import classification_report
from keras.metrics import MeanIoU
import joblib
from sklearn.model_selection import train_test_split
from scipy.stats import mode

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# SVM

# Import training data

In [6]:
# Open the TIFF file
dataset = gdal.Open("/content/drive/MyDrive/Colab Notebooks/Revision/clipped_post.tif", gdal.GA_ReadOnly)

# Read the first band (index starts at 1)
band9 = dataset.GetRasterBand(9)
band9 = band9.ReadAsArray()
# train = gpd.read_file(r"C:\Document\University\2023_Spring_Junior\GEOG_491\Dataset\August_Complex\Raw_testing\Post\matched_post.tif")
band9=band9.reshape(44236800, 1)
band8 = dataset.GetRasterBand(8)
band8 = band8.ReadAsArray()
band8=band8.reshape(44236800, 1)
band7 = dataset.GetRasterBand(7)
band7 = band7.ReadAsArray()
band7=band7.reshape(44236800, 1)
# print(band7.shape, band8.shape, band9.shape)

X_train=np.concatenate((band9,band8,band7),axis=1)
print(X_train.shape)

dataset = gdal.Open("/content/drive/MyDrive/Colab Notebooks/Revision/python_train_dnbr.tif", gdal.GA_ReadOnly)
# Read the first band (index starts at 1)
band = dataset.GetRasterBand(1)
band = band.ReadAsArray()
y_train=band.reshape(-1)
print(y_train.shape)

(44236800, 3)
(44236800,)


# Model training

In [10]:
# Initialize the SVM model
model1 = LinearSVC()

# Train the model
classification = model1.fit(X_train.astype(np.float64),y_train.astype(np.float64))

# Import testing data

In [11]:
# Open the TIFF file
dataset = gdal.Open("/content/drive/MyDrive/Colab Notebooks/Revision/matched_post.tif", gdal.GA_ReadOnly)

# Read the first band (index starts at 1)
band9 = dataset.GetRasterBand(9)
band9 = band9.ReadAsArray()
band9=band9.reshape(2359296, 1)
band8 = dataset.GetRasterBand(8)
band8 = band8.ReadAsArray()
band8=band8.reshape(2359296, 1)
band7 = dataset.GetRasterBand(7)
band7 = band7.ReadAsArray()
band7=band7.reshape(2359296, 1)
# print(band7.shape, band8.shape, band9.shape)

X_test=np.concatenate((band9,band8,band7),axis=1)
print(X_test.shape)

dataset = gdal.Open("/content/drive/MyDrive/Colab Notebooks/Revision/python_test_dnbr.tif", gdal.GA_ReadOnly)
# Read the first band (index starts at 1)
band = dataset.GetRasterBand(1)
band = band.ReadAsArray()
y_test=band.reshape(-1)
print(y_test.shape)

(2359296, 3)
(2359296,)


# Testing

In [12]:
predictions = model1.predict(X_test)

In [13]:
accuracy = metrics.accuracy_score(y_test, predictions)
print("SVM accracy is", accuracy)
cm = metrics.confusion_matrix(y_test, predictions)
print(cm)
print(classification_report(y_test, predictions, digits=4))

IOU_keras = MeanIoU(num_classes=4)
IOU_keras.update_state(y_test, predictions)
print("Mean IoU = ", IOU_keras.result().numpy())
values = np.array(IOU_keras.get_weights()).reshape(4, 4)
class1_IoU = values[0,0]/(values[0,0] + values[0,1] + values[0,2] + values[0,3] + values[1,0] + values[2,0] + values[3,0])
class2_IoU = values[1,1]/(values[1,1] + values[1,0] + values[1,2] + values[1,3] + values[0,1] + values[2,1] + values[3,1])
class3_IoU = values[2,2]/(values[2,2] + values[2,0] + values[2,1] + values[2,3] + values[0,2] + values[1,2] + values[3,2])
class4_IoU = values[3,3]/(values[3,3] + values[3,0] + values[3,1] + values[3,2] + values[0,3] + values[1,3] + values[2,3])
print("class1 IoU = ", class1_IoU)
print("class2 IoU = ", class2_IoU)
print("class3 IoU = ", class3_IoU)
print("class4 IoU = ", class4_IoU)

from sklearn.metrics import cohen_kappa_score
kappa_score = cohen_kappa_score(y_test, predictions)
print("Kappa:", kappa_score)

SVM accracy is 0.7519904242621528
[[1633841       0    1068    5333]
 [ 312298       0    7970   30524]
 [ 142197       0    4086   51577]
 [  32153       0    2008  136241]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0     0.7705    0.9961    0.8689   1640242
           1     0.0000    0.0000    0.0000    350792
           2     0.2700    0.0207    0.0384    197860
           3     0.6091    0.7995    0.6914    170402

    accuracy                         0.7520   2359296
   macro avg     0.4124    0.4541    0.3997   2359296
weighted avg     0.6023    0.7520    0.6572   2359296

Mean IoU =  0.329036
class1 IoU =  0.7681831
class2 IoU =  0.0
class3 IoU =  0.019559037
class4 IoU =  0.5284018
Kappa: 0.3256209770743177
